In [74]:
import pandas as pd
import datetime
import warnings
import copy
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import http.client 
import requests
from sklearn.linear_model import LinearRegression
import statsmodels

In [47]:
request_url = "https://api.gateway.attomdata.com/v4/transaction/salestrend?geoIdV4=12c8e6811019260f3895b7be715f081c&interval=yearly&startyear=2003&endyear=2022"

In [48]:
conn = http.client.HTTPSConnection("api.gateway.attomdata.com") 

headers = { 
    'accept': "application/json", 
    'apikey': "8acc04a06164433b58ed0d8b1ed0c08b", 
} 

params = {
    "geoIdV4": "12c8e6811019260f3895b7be715f081c",
    "startyear": "2003",
    "endyear":"2022"	
}

housing_req = requests.get(request_url, params = params, headers = headers)

housing_req.raise_for_status

<bound method Response.raise_for_status of <Response [200]>>

In [49]:
location_request = requests.get("https://api.gateway.attomdata.com/v4/location/lookup?geographyTypeAbbreviation=CI&name=San Francisco, CA")
location_request = requests.get("https://api.gateway.attomdata.com/v4/location/lookup?geographyTypeAbbreviation=CI&name=San Francisco, CA", headers = headers)
location_request.raise_for_status
locations_json = location_request.json()

In [50]:
locations_json["geographies"]

[{'geoIdV4': '12c8e6811019260f3895b7be715f081c',
  'geographyName': 'San Francisco, CA',
  'geographyTypeName': 'Postal City',
  'geographyTypeAbbreviation': 'CI',
  'details': {'name': 'San Francisco',
   'totalResidentCount': 367111,
   'multiFamilyDeliveryUnitCount': 190975,
   'singleFamilyDeliveryUnitCount': 169378,
   'poBoxCount': 6758,
   'businessCount': 39402,
   'areaSquareMiles': 52.7570254934379,
   'longitude': -122.445362,
   'latitude': 37.760434}},
 {'geoIdV4': 'b9caab2b3889a2caadddc62013721fd0',
  'geographyName': 'South San Francisco, CA',
  'geographyTypeName': 'Postal City',
  'geographyTypeAbbreviation': 'CI',
  'details': {'name': 'South San Francisco',
   'totalResidentCount': 21414,
   'multiFamilyDeliveryUnitCount': 5092,
   'singleFamilyDeliveryUnitCount': 16322,
   'poBoxCount': 0,
   'businessCount': 2405,
   'areaSquareMiles': 10.4541633181068,
   'longitude': -122.417006,
   'latitude': 37.656305}}]

In [51]:
housing_json = housing_req.json()

In [52]:
housing_json["salesTrends"]

[{'location': {'geoIdV4': '12c8e6811019260f3895b7be715f081c',
   'geographyName': 'San Francisco, CA'},
  'dateRange': {'interval': 'yearly', 'start': '2003', 'end': '2003'},
  'salesTrend': {'homeSaleCount': 9144,
   'avgSalePrice': 648759,
   'medSalePrice': 560000},
  'vintage': {'pubDate': '2022-02-12'}},
 {'location': {'geoIdV4': '12c8e6811019260f3895b7be715f081c',
   'geographyName': 'San Francisco, CA'},
  'dateRange': {'interval': 'yearly', 'start': '2004', 'end': '2004'},
  'salesTrend': {'homeSaleCount': 9715,
   'avgSalePrice': 788171,
   'medSalePrice': 675000},
  'vintage': {'pubDate': '2022-02-12'}},
 {'location': {'geoIdV4': '12c8e6811019260f3895b7be715f081c',
   'geographyName': 'San Francisco, CA'},
  'dateRange': {'interval': 'yearly', 'start': '2005', 'end': '2005'},
  'salesTrend': {'homeSaleCount': 9413,
   'avgSalePrice': 918877,
   'medSalePrice': 769000},
  'vintage': {'pubDate': '2022-02-12'}},
 {'location': {'geoIdV4': '12c8e6811019260f3895b7be715f081c',
   'g

In [53]:
def get_housing_df():
    all_years = pd.DataFrame()
    for i in housing_json["salesTrends"]:
        df = pd.DataFrame({"year": [i['dateRange']["start"]], "mean_sale": [i['salesTrend']['avgSalePrice']],
                            "median_sale": [i['salesTrend']['medSalePrice']]})
        all_years = pd.concat([all_years, df])
    return all_years

In [54]:
housing_df = get_housing_df()

In [55]:
housing_df = housing_df.apply(pd.to_numeric)

In [56]:
housing_df

,year,mean_sale,median_sale
0,2003,648759,560000
0,2004,788171,675000
0,2005,918877,769000
0,2006,939626,785000
0,2007,984063,800000
0,2008,936984,760000
0,2009,826873,675000
0,2010,885453,699000
0,2011,906313,682000
0,2012,977190,750000


In [57]:
housing_df.dtypes

year           int64
mean_sale      int64
median_sale    int64
dtype: object

In [58]:
housing_df.to_csv('housing.csv')